<a href="https://colab.research.google.com/github/GeorgeTelles/Extrair_analisar_fundos_investimento./blob/main/Extrair_e_analisar_fundos_de_investimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>


# **Extrair e analisar dados de fundos de investimento**

Esse algoritmo em Python tem como objetivo obter dados de fundos de investimento direto da fonte: o portal de dados da CVM.

Vou processar estes dados para posteriormente analisá-los.

Quero responder algumas perguntas importantes, como:

1. Quais fundos do Brasil tem o maior PL (patrimônio líquido)?
2. Quais fundos tem mais cotistas?
3. Quais os 10 fundos que teviveram as maiores altas?
4. Quais os 10 fundos que teviveram as maiores baixas?

# **0. Noções básicas**

### O que é um fundo de investimento?

**O que é um Fundo de Investimentos?**
Um fundo de investimentos funciona como uma espécie de condomínio.

Dentro de um fundo temos vários segmentos:

Existe a empresa administradora do fundo, o gestor, os profissionais que fazem a auditoria sobre os números do fundo e, por fim, mas não menos importante, os investidores, ou nesse caso, os cotistas.

O fundo de investimento parte da prerrogativa: Porque investir sozinho, se o investidor pode fazer isso com mais pessoas, utilizando muito mais recursos e contratando um profissional ultra qualificado para cuidar de todo esse dinheiro?

É por isso que, em grande parte, os fundos funcionam como condomínios.

Fonte: https://maisretorno.com/portal/termos/f/fundo-de-investimentos

Fonte: https://blog.yubb.com.br/entenda-os-fundos-de-investimentos/

Ferramentas importantes para ajudar a analisar e comparar fundos de investimento

Portal de dados Anbima
https://data.anbima.com.br/

Mais Retorno
https://maisretorno.com/comparacao-fundos



1. Identificar os sites que farão parte da busca (cotas e cadastral)
2. Iniciar extração dos dados
3. Obter e tratar os dados
4. Juntar os dois e começar as análises
5. Responder questionamentos do projeto

### Qual o processo?

Vou começar a extração explorando as páginas da CVM que tem as informações que serão extraídas.

Para dados de desempenho dos fundos:

https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/

Para dados cadastrais dos fundos:

http://dados.cvm.gov.br/dados/FI/CAD/DADOS/

dados abertos da CVM:

https://dados.cvm.gov.br/

# **1. Extração e processamento dos dados**

## 1.1. Importação das bibliotecas

In [ ]:
import zipfile
import io
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:.2f}'.format

## 1.2. Captura do primeiro conjunto de dados

In [ ]:
#O nome do arquivo precisa conter o ano e mês mais recentes ou do mês anterior, neste exemplo 2024/08
arquivo = 'inf_diario_fi_202408.csv'

link = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202408.zip'

In [ ]:
r = requests.get(link)

In [ ]:
zf = zipfile.ZipFile(io.BytesIO(r.content))
arquivo_fi = zf.open(arquivo)

In [ ]:
linhas = arquivo_fi.readlines()
linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
linhas = [i.split(';') for i in linhas]

In [ ]:
df = pd.DataFrame(linhas, columns = linhas[0])
informes_diarios = df[1:].reset_index()
informes_diarios[['VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST']] = informes_diarios[['VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST']].apply(pd.to_numeric)
informes_diarios

## 1.3. Captura do segundo conjunto de dados

In [ ]:
url = "http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv"

In [ ]:
cadastral = pd.read_csv(url, sep = ';', encoding = 'ISO-8859-1')
cadastral = cadastral.drop_duplicates(subset='CNPJ_FUNDO')

**Agora temos 2 conjuntos de dados: o Data Frame "cadastral" e o "informes_diarios", vou juntar os 2 em um só**

In [ ]:
# Mesclar os dataframes
dados_fundos = pd.merge(informes_diarios, cadastral, on='CNPJ_FUNDO', how='left')

**Tratamento do DataFrame**

In [ ]:
dados_fundos = dados_fundos[dados_fundos['VL_QUOTA'] != 0]
dados_fundos = dados_fundos[dados_fundos['NR_COTST'] > 150]

In [ ]:
qtd_fundos = dados_fundos['CNPJ_FUNDO'].nunique()
print(f"Temos {qtd_fundos} Fundos com no minimo 150 cotistas")

# **2. Análises com dados cadastrais**

## 2.1. 10 Fundos com maiores Patrimônio Liquidos

In [ ]:
top_10_pl = dados_fundos[dados_fundos['DT_COMPTC'] == '2024-08-01'] \
    .sort_values('VL_PATRIM_LIQ_x', ascending=False) \
    .head(10) \
    .loc[:, ['CNPJ_FUNDO', 'DENOM_SOCIAL', 'CLASSE', 'VL_PATRIM_LIQ_x']]

top_10_pl

## 2.2. fundos com maiores numeros de cotistas

In [ ]:
top_10_cotistas = dados_fundos[dados_fundos['DT_COMPTC'] == '2024-08-01'] \
    .sort_values('NR_COTST', ascending=False) \
    .head(10) \
    .loc[:, ['CNPJ_FUNDO', 'DENOM_SOCIAL', 'CLASSE', 'NR_COTST']]

top_10_cotistas

## 2.2. Os 10 fundos que mais valorizaram e os 10 fundos que mais desvalorizaram

**Calculando os retornos**

In [ ]:
dados_fundos = dados_fundos.copy()
dados_fundos['RETORNO'] = dados_fundos.groupby('CNPJ_FUNDO')['VL_QUOTA'].pct_change() * 100
dados_fundos = dados_fundos.dropna(subset=['RETORNO'])
pass


In [ ]:
dados_fundos = dados_fundos.copy()
dados_fundos['RETORNO_ACUMULADO'] = dados_fundos.groupby('CNPJ_FUNDO')['RETORNO'].cumsum()
dados_fundos = dados_fundos.dropna(subset=['RETORNO_ACUMULADO'])
retorno_total = dados_fundos.groupby('DENOM_SOCIAL')['RETORNO_ACUMULADO'].last()

**10 Maiores altas**

In [ ]:
top_10_altas = retorno_total.sort_values(ascending=False).head(10)
top_10_altas_df = top_10_altas.reset_index()

top_10_altas_df.columns = ['DENOM_SOCIAL', 'RETORNO_ACUMULADO']
top_10_altas_df['INDEX'] = range(1, len(top_10_altas_df) + 1)


top_10_altas_df = top_10_altas_df.set_index('INDEX')

top_10_altas_df

**10 Maiores Baixas**

In [ ]:
# Obter os 10 fundos com os menores retornos acumulados
top_10_baixas = retorno_total.sort_values(ascending=True).head(10)

# Converter para DataFrame e adicionar coluna de índice
top_10_baixas_df = top_10_baixas.reset_index()
top_10_baixas_df.columns = ['DENOM_SOCIAL', 'RETORNO_ACUMULADO']
top_10_baixas_df['INDEX'] = range(1, len(top_10_baixas_df) + 1)

# Definir a nova coluna 'INDEX' como índice
top_10_baixas_df = top_10_baixas_df.set_index('INDEX')

# Mostrar o DataFrame com o novo índice
top_10_baixas_df
